### Importing Libraries


In [7]:
import os
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [8]:
def get_dataloaders(data_dir, batch_size=32):
    """
    DataLoader for 32x32 grayscale images as required by LeNet-5
    """
    transform = transforms.Compose(
        [
            transforms.Resize((32, 32)),  # Resize to 32x32 
            transforms.Grayscale(
                num_output_channels=1
            ), 
            transforms.ToTensor(),  
        ]
    )

    train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform)
    val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform)
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, len(train_dataset.classes)

In [9]:
class LeNet5Model(nn.Module):
    """
    LeNet-5 Architecture for 32x32x1 grayscale input and 10-class classification.
    Total parameters: ~61,706
    """

    def __init__(self, num_classes=10):
        super(LeNet5Model, self).__init__()

        # Feature extraction layers
        self.features = nn.Sequential(
            # C1: Convolutional Layer
            # Input: 1x32x32 → Output: 6x28x28 
            nn.Conv2d(
                in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0
            ),
            nn.Tanh(),
            # S2: Average Pooling Layer
            # Input: 6x28x28 → Output: 6x14x14
            nn.AvgPool2d(kernel_size=2, stride=2),
            # C3: Convolutional Layer
            # Input: 6x14x14 → Output: 16x10x10 
            nn.Conv2d(
                in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0
            ),
            nn.Tanh(),
            # S4: Average Pooling Layer
            # Input: 16x10x10 → Output: 16x5x5
            nn.AvgPool2d(kernel_size=2, stride=2),
        )

        # Fully connected classification layers
        self.classifier = nn.Sequential(
            # Flatten: 16x5x5 = 400 features
            # FC1: 400 → 120
            nn.Linear(16 * 5 * 5, 120),
            nn.Tanh(),
            # FC2: 120 → 84
            nn.Linear(120, 84),
            nn.Tanh(),
            # Output: 84 → num_classes
            nn.Linear(84, num_classes),
        )

    def forward(self, x):
        """
        Forward pass through the network
        Args:
            x: Input tensor of shape (batch_size, 1, 32, 32)
        Returns:
            Output tensor of shape (batch_size, num_classes)
        """
        x = self.features(x)
        x = x.view(x.size(0), -1) 
        x = self.classifier(x)
        return x

In [10]:
def count_parameters(model):
    """Count total trainable parameters in the model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def lenet5_from_scratch():
    data_dir = r"E:\Semester 7\Selected Topics in Artificial Intelligence-1\Project\Repo\Dangerous-Farm-Insects-Classification\data\processed\farm_insects\splits"
    num_epochs = 10
    lr = 1e-4

    # Load Data
    train_loader, val_loader, test_loader, num_classes = get_dataloaders(data_dir)

    # Initialize Model
    model = LeNet5Model(num_classes=num_classes)
    
    # Verify parameter count
    total_params = count_parameters(model)
    print(f"\nLeNet-5 Model Initialized")
    print(f"Total trainable parameters: {total_params:,}")
    print(f"Expected: ~61,706 (for 10 classes)\n")

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()  
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in tqdm(
            train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"
        ):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        train_loss = running_loss / len(train_loader)

        # Validation Loop
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in tqdm(
                val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"
            ):
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = outputs.max(1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        val_loss = val_loss / len(val_loader)

        print(f"\nEpoch {epoch+1}/{num_epochs} Summary:")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
        print("-" * 60)

    print("\nTraining complete!")

    # Testing 
    print("\nEvaluating on Test Set...")
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="[Test]"):
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute Metrics
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
    rec = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
    f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)

    print("\nTest Set Metrics:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1-score:  {f1:.4f}")

In [12]:
lenet5_from_scratch()


LeNet-5 Model Initialized
Total trainable parameters: 62,131
Expected: ~61,706 (for 10 classes)



Epoch 1/10 [Train]:  50%|█████     | 16/32 [00:11<00:12,  1.26it/s]c:\Python311\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1/10 [Val]: 100%|██████████| 8/8 [00:11<00:00,  1.50s/it]



Epoch 1/10 Summary:
  Train Loss: 2.7083 | Train Acc: 6.94%
  Val Loss:   2.7066 | Val Acc:   7.11%
------------------------------------------------------------


Epoch 2/10 [Val]: 100%|██████████| 8/8 [00:05<00:00,  1.37it/s]



Epoch 2/10 Summary:
  Train Loss: 2.7046 | Train Acc: 7.93%
  Val Loss:   2.7042 | Val Acc:   9.09%
------------------------------------------------------------


...


Epoch 9/10 Summary:
  Train Loss: 2.6544 | Train Acc: 11.50%
  Val Loss:   2.6681 | Val Acc:   9.88%
------------------------------------------------------------


Epoch 10/10 [Val]: 100%|██████████| 8/8 [00:11<00:00,  1.47s/it]



Epoch 10/10 Summary:
  Train Loss: 2.6436 | Train Acc: 11.79%
  Val Loss:   2.6627 | Val Acc:   11.07%
------------------------------------------------------------

Training complete!

Evaluating on Test Set...


[Test]: 100%|██████████| 10/10 [00:16<00:00,  1.62s/it]


Test Set Metrics:
Accuracy:  0.1203
Precision: 0.0874
Recall:    0.1203
F1-score:  0.0909
